# Aggregation and Grouping

An essential piece of data analysis is summarizing data. This often involves a "group by" operation, where you split data into groups, apply some function to each group (like an aggregation), and then combine the results.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# For this notebook, we'll use the Planets dataset
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

In [ ]:
planets.head()

## Simple Aggregation in Pandas

A `DataFrame` can have aggregations computed over its columns.

In [ ]:
# Create a simple DataFrame
df = pd.DataFrame([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9],
                   [np.nan, np.nan, np.nan]],
                  columns=['A', 'B', 'C'])

# Aggregate functions
df.mean(axis=0) # Get mean of each column

In [ ]:
# The describe() method computes several common aggregates
planets.dropna().describe()

## GroupBy: Split, Apply, Combine

The real power of these operations comes with the `groupby()` method. It follows a three-step process:
1.  **Split**: Break up the data into groups based on some key.
2.  **Apply**: Compute some function (usually an aggregate, transformation, or filter) within the individual groups.
3.  **Combine**: Merge the results of these operations into an output array.

In [ ]:
# `groupby` takes columns for `by` parameter by default
# to specify grouping by index label, use `level` parameter
planets.groupby('method')['orbital_period'].median()

The `groupby()` object is very flexible. You can iterate over it to see the groups. It creates invisible boundaries between each specified group based on a key value so that rows with the same value are grouped together. This works well for aggregating statistics on an unsorted dataset.

In [ ]:
# Iterating over groups
for (method, group) in planets.groupby('method'):
    print(f"{method:30s} shape={group.shape}")

### Aggregate, Filter, Transform, Apply

The `groupby` object has several useful methods.

**Aggregation**: The `aggregate()` method can take a string, a function, or a list thereof and compute all the aggregates at once.

In [ ]:
planets.groupby('method')['distance'].aggregate(['min', 'median', 'max'])

**Filtering**: A filtering operation allows you to drop data based on the group properties. For example, we might want to keep all groups in which the standard deviation is larger than some critical value.

In [ ]:
def filter_func(x):
    return x['distance'].std() > 50

planets.groupby('method').filter(filter_func).groupby('method').size()
# filter iterates on the smaller grouped dataframes from groupby and then returns a concatenated version of "passing" dataframes
# .size() is useful for counting the number of elements within a specific group

**Transformation**: While aggregation must return a reduced version of the data, transformation can return some transformed version of the full data to recombine. A common example is to center the data by subtracting the group-wise mean.

In [ ]:
planets.groupby('method').transform(lambda x: x - x.mean()).head()

# Performs element-wise operation, often using lambda function
# Returns same shape as original Series/DataFrame

**Apply**: The `apply()` method lets you apply an arbitrary function to the group results. The function should take a `DataFrame` and return either a Pandas object (e.g., `DataFrame`, `Series`) or a scalar.

In [ ]:
# Example: Get the second-largest distance for each method
def get_second_largest_dist(x):
    return x.nlargest(2, 'distance')

planets.groupby('method').apply(get_second_largest_dist, include_groups=False)
# Best practice includes either selecting the columns that the function is being applied to
# or specifying include_groups=False which prevents grouping column (from groupby) from being added
# to the DataFrame passed to the applied function

# This is because when smaller groupwise dfs are passed to the applied function, the grouping key is still included as a column (e.g., Method: a, a, a, a, a...). This causes issues if the applied function is only supposed to operate on numeric values and the grouping column has string values, or something like that.

# Additional Ways to Specify Keys for GroupBy

In the previous sections, we saw how to use a single column name to specify the groups for a `groupby` operation. Here, we'll explore other ways to define groups.

In [ ]:
# We'll use the following DataFrame for our examples:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': np.random.randint(0, 10, 6)},
                  columns = ['key', 'data1', 'data2'])
df

## 1. A List, Array, Series, or Index

You can provide a list, array, or Series with a length matching the number of rows in the DataFrame to specify the grouping keys.

In [ ]:
L = [0, 1, 0, 1, 2, 0]
# list is ordered by row indices; values as grouping keys may be anything hashable
df.groupby(L).sum(numeric_only=True)

In [ ]:
# This is equivalent to the `df.groupby('key')` we've seen before, but more verbose. It is essentially passing in a `pd.Series` object to groupby, instead of specifying a column name.
df.groupby(df['key']).sum()

## 2. A Dictionary or Series Mapping the Index to a Group

Another way to specify groups is to provide a dictionary or a Series that maps the index values to group keys.

In [ ]:
# First, let's set the 'key' column as the index.
print(df)
df2 = df.set_index('key')
df2

In [ ]:
# Now, let's define a mapping from the index values to groups.
mapping = {'A': 'vowel', 'B': 'consonant'}
df2.groupby(mapping).max() #type: ignore

# The keys of the dictionary map to unique index values in the dataframe, which in this case is `key`.
# The values of the dictionary map to the grouping keys for `groupby`.
# Any index values that are excluded are excluded from the grouping and subsequent aggregation function.

## 3. Any Python Function

You can also pass any Python function that takes the index value as input and returns the group as output.

In [ ]:
# For example, we can group by the lowercase version of the index values:
df2.groupby(str.lower).mean()

## 4. A List of Valid Keys

Any of the preceding key choices can be combined to group on a multi-index.

In [ ]:
df2.groupby([str.lower, mapping]).mean()

## Grouping Example

Let's see a more complex example that combines these concepts. We'll use a dataset of discovered exoplanets.

In [ ]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
result = planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

result

This example demonstrates the power of combining different grouping strategies. In just a few lines of code, we can get a summary of how many planets were discovered by each method in each decade.

# Pivot Tables

`GroupBy` allows us to explore relationships in a dataset. A pivot table is a similar operation that takes column-wise data and groups it into a two-dimensional table, providing a multidimensional summary.

Think of pivot tables as a multidimensional version of `GroupBy` aggregation. You split-apply-combine across a two-dimensional grid instead of a one-dimensional index.

## Motivating Pivot Tables

We'll use the Titanic dataset from the Seaborn library for these examples.

In [ ]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic.head()

## Pivot Tables by Hand

To understand the data, we can group it by different attributes. For example, let's look at the survival rate by gender.

In [ ]:
titanic.groupby('sex')[['survived']].mean()

This shows that about 74% of females survived, while only about 19% of males survived.

Now, let's look at survival by both sex and class.

In [ ]:
titanic.groupby(['sex', 'class'], observed=True)['survived'].mean().unstack()

This gives us a better idea of how sex and class affected survival, but the code is a bit complex. The `pivot_table` method provides a more readable way to do this.

## Pivot Table Syntax

Here's the equivalent of the previous operation using `pivot_table`:

In [ ]:
print(titanic)
titanic.pivot_table('survived', index='sex', columns='class', observed=True) # default aggfunc is mean

This is much easier to read and produces the same result.

## Multilevel Pivot Tables

We can also specify multiple levels for grouping. Let's add age as a third dimension.

In [ ]:
age = pd.cut(titanic['age'], [0, 18, 80])
titanic.pivot_table('survived', ['sex', age], 'class', observed=False) # multiple indexing

We can also work with columns. Let's add the fare paid.

In [ ]:
fare = pd.qcut(titanic['fare'], 2)
titanic.pivot_table('survived', ['sex', age], [fare, 'class'], observed=False)

## Additional Pivot Table Options

The `pivot_table` method has several other options. The `aggfunc` keyword controls the aggregation type (default is 'mean').

In [ ]:
titanic.pivot_table(index='sex', columns='class',
                   aggfunc={'survived':'sum', 'fare':'mean'},
                   observed=False)

The `margins` keyword is useful for computing totals along each grouping.

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class', margins=True, observed=False)

## Example: Birthrate Data

Let's look at the US birthrate data from the CDC.

In [ ]:
births = pd.read_csv('../data/births.csv')
print(births)

# Add a decade column and look at male and female births by decade.
births['decade'] = 10 * (births['year'] // 10)
births.pivot_table('births', index='decade', columns='gender', aggfunc='sum')

We can see that male births outnumber female births in every decade. Let's visualize this.

In [ ]:
import matplotlib.pyplot as plt
births.pivot_table('births', index='year', columns='gender', aggfunc='sum').plot()
plt.ylabel('total births per year');

### Further Data Exploration

Let's clean the data by removing outliers.

In [ ]:
quartiles = np.percentile(births['births'], [25, 50, 75])
mu = quartiles[1]
sig = 0.74 * (quartiles[2] - quartiles[0])
births = births.query('(births > @mu - 5 * @sig) & (births < @mu + 5 * @sig)')

In [ ]:
# Set the 'day' column to integer type.
births['day'] = births['day'].astype(int)

In [ ]:
# Create a datetime index from the year, month, and day.
print(births)
births.index = pd.to_datetime(births[['year', 'month', 'day']], errors='coerce')
births['dayofweek'] = births.index.dayofweek
births

Plot births by weekday for several decades.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
births.pivot_table('births', index='dayofweek', columns='decade', aggfunc='mean').plot()
plt.gca().set_xticklabels(['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun'])
plt.ylabel('mean births by day');

It appears that births are less common on weekends.

Let's plot the mean number of births by the day of the year.

In [ ]:
births.index

In [ ]:
# Calculates the births for each calendar day, averaged throughout the years
births_by_date = births.pivot_table('births', index=[births.index.month, births.index.day]) #type: ignore

# note: year here is entirely inaccurate; only for the purposes of simple display
births_by_date.index = [pd.Timestamp(2012, month, day) for (month, day) in births_by_date.index] #type: ignore

print(births_by_date)
fig, ax = plt.subplots(figsize=(12, 4))
births_by_date.plot(ax=ax)

The plot shows a dip in birthrate on US holidays.